In [1]:
import torch
import torch.nn as nn
from preprocessing import NextWordDataset

/home2/sanika/miniconda3/envs/smai/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
path = '/scratch/sanika/content/sample_data/fulldataset_dedup_final.txt'

In [4]:
dataset = NextWordDataset(path=path, end=10000)

Tokenized data


In [5]:
words, targets, vocab, character_vocab = dataset.format(dataset.sentences, 5)

In [7]:
words[0], targets[0]

([tensor([48, 41, 19,  1, 41, 33,  0,  0,  0,  0]),
  tensor([10, 29, 32, 19, 15,  7, 51,  0,  0,  0]),
  tensor([14,  7, 15,  0,  0,  0,  0,  0,  0,  0]),
  tensor([14, 47, 41, 19, 29, 48, 33, 51,  0,  0]),
  tensor([10, 51,  7, 29, 43, 51,  0,  0,  0,  0])],
 tensor(70))

In [6]:
# make the character embedding and convolutional layer with max pooling
class CharCNN(nn.Module):
    def __init__(self, character_embedding_size, num_filters, kernel_size, max_word_length, char_vocab_size, word_embedding_dim, device = device):
        super(CharCNN, self).__init__()
        self.char_embedding = nn.Embedding(char_vocab_size, character_embedding_size)
        self.conv_layers = nn.ModuleList([nn.Conv1d(character_embedding_size
                                                    , num_filters, kernel_size) for _ in range(max_word_length - kernel_size + 1)])
        self.fc = nn.Linear(num_filters * (max_word_length - kernel_size + 1), word_embedding_dim)
    def forward(self, x):
        # x is a batch of words. Each word is a list of characters (batch_size, max_word_length)
        # first, we convert the characters to embeddings
        x = self.char_embedding(x) # (batch_size, max_word_length, character_embedding_size)
        x = x.permute(0, 2, 1) # (batch_size, character_embedding_size, max_word_length)
        # now we run the convolutional layers
        x = [conv(x) for conv in self.conv_layers]
        # now we max pool
        x = [torch.max(torch.relu(conv), dim = 2)[0] for conv in x]
        # now we concatenate the results
        x = torch.cat(x, dim = 1) # (batch_size, num_filters * (max_word_length - kernel_size + 1))
        # finally, we run the fully connected layer
        x = self.fc(x)
        return x 

# ELMo part
class ELMo(nn.Module):
    def __init__(self, cnn_config, elmo_config, char_vocab_size):
        # input to this is a batch of sentences. Each sentence is a list of words. Each word is a list of characters.
        super(ELMo, self).__init__()
        # first, we convert the token to a representation using character embeddings
        self.char_cnn = CharCNN(cnn_config['character_embedding_size'], 
                                cnn_config['num_filters'], 
                                cnn_config['kernel_size'], 
                                cnn_config['max_word_length'], 
                                cnn_config['char_vocab_size'],
                                elmo_config['word_embedding_dim'],
                                device = device)
        self.forward_lstm = nn.LSTM(elmo_config['word_embedding_dim'], int(elmo_config['word_embedding_dim']/2), 
                                    1, bidirectional = False)
        self.backward_lstm = nn.LSTM(elmo_config['word_embedding_dim'], int(elmo_config['word_embedding_dim']/2),
                                    1, bidirectional = False)
        # based on the number of layers as passed in the argument, sequentially have that many layers
        self.forward_lstms = nn.ModuleList([self.forward_lstm for _ in range(elmo_config['num_layers'])])
        self.backward_lstms = nn.ModuleList([self.backward_lstm for _ in range(elmo_config['num_layers'])])
        self.num_layers = elmo_config['num_layers']
        self.fc = nn.Linear(elmo_config['word_embedding_dim'], elmo_config['vocab_size'])
        
    def forward(self, x):
        # character cnn
        x = [self.char_cnn(sentence) for sentence in x]
        # lstm1
        x = torch.stack(x, dim=1) 
        x = x.permute(1, 0, 2) 
        
        lstm_output = x
        for i in range(self.num_layers):
            forward_lstm_output, _ = self.forward_lstms[i](lstm_output)
            backward_lstm_output, _ = self.backward_lstms[i](torch.flip(lstm_output, [1]))
            backward_lstm_output = torch.flip(backward_lstm_output, [1])
            lstm_output = torch.cat((forward_lstm_output, backward_lstm_output), dim = 2)
        
        x = torch.mean(lstm_output, dim = 1)
        x = self.fc(x)
        return x, lstm_output
     

In [7]:
model = ELMo(cnn_config = {'character_embedding_size': 16, 
                           'num_filters': 32, 
                           'kernel_size': 5, 
                           'max_word_length': 16, 
                           'char_vocab_size': len(character_vocab)}, 
             elmo_config = {'num_layers': 3,
                            'word_embedding_dim': 300,
                            'vocab_size': 1000}, 
             char_vocab_size = len(character_vocab)).to(device)


In [11]:
for i, (x, y) in enumerate(train_loader):
    x = x[0]
    x = x.to(device)
    # model(x)
    # break
    print(x)
    break

ValueError: not enough values to unpack (expected 2, got 1)